In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize
from sklearn.metrics import r2_score
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# --------------------- 1. Model Definitions ----------------------

def langmuir(Ce, qm, KL):
    """Langmuir isotherm."""
    return (qm * KL * Ce) / (1 + KL * Ce)

def freundlich(Ce, KF, n):
    """Freundlich isotherm."""
    return KF * Ce ** (1 / n)

def temkin(Ce, A, B):
    """Temkin isotherm."""
    return B * np.log(A * Ce + 1e-12)

def sips(Ce, qm, Ks, n):
    """Corrected Sips (Langmuir–Freundlich) isotherm."""
    x = Ks * Ce
    xn = np.power(x, n, where=x > 0)
    return qm * xn / (1 + xn + 1e-12)

# --------------------- 2. Helper Functions ----------------------

def create_jacobian(func, Ce, params):
    """Finite-difference Jacobian with relative perturbation."""
    eps = np.sqrt(np.finfo(float).eps)
    jac = np.zeros((len(Ce), len(params)))
    for i, p in enumerate(params):
        h = eps * max(abs(p), 1.0)
        p_plus = params.copy(); p_plus[i] += h
        p_minus = params.copy(); p_minus[i] -= h
        jac[:, i] = (func(Ce, *p_plus) - func(Ce, *p_minus)) / (2 * h)
    return jac

def calculate_metrics(y, yhat, Np, pcov=None):
    """Calculates model performance metrics using formulas from the paper."""
    Nc = len(y)  # Eq. (2,3,4,6,7): N_C = sample size
    rss = np.sum((y - yhat) ** 2)  # Eq. (3): J = sum(epsilon_i^2)
    sigma2 = rss / Nc if rss > 0 else 1e-12  # Eq. (4): sigma_Y^2 = J_min / N_C
    nll = (rss / sigma2) + Nc * np.log(2 * np.pi * sigma2)  # Eq. (2): NLL = J / sigma_Y^2 + N_C ln(2 pi sigma_Y^2)
    aic = nll + 2 * Np  # Eq. (6): AIC = NLL + 2 N_P
    bic = nll + Np * np.log(Nc)  # Eq. (7): BIC = NLL + N_P ln(N_C)
    kic = np.inf  # Default if no pcov
    if pcov is not None:
        det_Q = np.linalg.det(pcov)
        if det_Q > 1e-100:  # Avoid log(0) or tiny values
            kic = nll - Np * np.log(2 * np.pi) - np.log(det_Q)  # Eq. (8): KIC = NLL - N_P ln(2 pi) - ln|Q|
    return nll, aic, bic, kic

# Equations:
# Eq. (3): J = Σ ε_i^2
# Eq. (4): σ_Y^2 = J_min / N_C
# Eq. (2): NLL = J / σ_Y^2 + N_C ln(2π σ_Y^2)
# Eq. (6): AIC = NLL + 2 N_P
# Eq. (7): BIC = NLL + N_P ln(N_C)
# Eq. (8): KIC = NLL - N_P ln(2π) - ln|Q|

def calculate_posterior_probabilities(ic_values):
    """
    Calculates posterior model probabilities following the explicit method of Equation (9) from the paper.
    """
    ic_values = np.asarray(ic_values)
    finite_ics = np.isfinite(ic_values)
    if not finite_ics.any():
        return np.zeros_like(ic_values)
    NM = len(ic_values)  # Number of models (N_M)
    prior_prob = 1 / NM  # Equal priors: p(M_k) = 1 / N_M
    delta_ic = ic_values - np.min(ic_values[finite_ics])  # Delta IC_k = IC_k - IC_min
    numerator_term = np.exp(-0.5 * delta_ic) * prior_prob  # exp(-1/2 Delta IC_k) * p(M_k)
    numerator_term[~finite_ics] = 0  # Zero for non-finite ICs
    denominator_sum = np.sum(numerator_term[finite_ics])  # Sum over j: exp(-1/2 Delta IC_j) * p(M_j)
    if denominator_sum == 0:
        return np.zeros_like(ic_values)
    posterior_probs = numerator_term / denominator_sum  # Eq. (9): Full posterior formula
    return posterior_probs

# Equation:
# Eq. (9): p(M_k|C*) = [exp(-0.5 ΔIC_k) * p(M_k)] / Σ_j [exp(-0.5 ΔIC_j) * p(M_j)]

def param_stats(popt, pcov):
    if pcov is None: return None, None
    sd = np.sqrt(np.diag(pcov))
    cv = np.abs(sd / popt) * 100
    return sd, cv

# ---------------------- 3. Main Analysis --------------------------

def analyze_isotherms(file_path, V=0.2, m=0.5, export_results=True):
    """Maximum-likelihood fitting & model discrimination (AIC/BIC/KIC)."""
    df = pd.read_excel(file_path)
    c0_vals = sorted({float(c.split("mgL_")[0]) for c in df.columns if "mgL_Rep" in c})
    Ce_vals, qe_vals = [], []
    for c0 in c0_vals:
        reps = [c for c in df.columns if f"{int(c0)}mgL_Rep" in c]
        raw_mean = df[reps].iloc[-1].mean()
        Ce = raw_mean * c0 if raw_mean < 1 else raw_mean
        qe = (c0 - Ce) * V / m
        Ce_vals.append(Ce)
        qe_vals.append(qe)
    Ce, qe = np.array(Ce_vals), np.array(qe_vals)

    models = {
        "Langmuir":   dict(func=langmuir,   p0=[40, 0.1],  bounds=([0, 0],       [1e3, 1e2]), names=["qm", "KL"], color="r", ls="-"),
        "Freundlich": dict(func=freundlich, p0=[10, 2],    bounds=([0, 0.1],     [1e4, 10]),  names=["KF", "n" ], color="b", ls="--"),
        "Temkin":     dict(func=temkin,     p0=[1, 5],     bounds=([0, 0],       [1e2, 1e3]), names=["A",  "B" ], color="g", ls="-.") ,
        "Sips":       dict(func=sips,       p0=[40, 0.02, 1], bounds=([1, 1e-4, 0.1], [500, 10, 5]), names=["qm","Ks","n"], color="m", ls=":"),
    }
    fig, ax_fit = plt.subplots(figsize=(8, 6))
    ax_fit.plot(Ce, qe, "ko", ms=8, label="Experimental")
    summary, param_table, cov_sheets = {}, [], {}

    for name, spec in models.items():
        func, p0, bounds = spec["func"], spec["p0"], spec["bounds"]
        try:
            popt, _ = curve_fit(func, Ce, qe, p0=p0, bounds=bounds, maxfev=10000)
            def nll(theta):
                res = qe - func(Ce, *theta)
                rss = np.sum(res**2)
                s2 = rss / len(qe) if rss > 0 else 1e-12
                return (rss / s2) + len(qe) * np.log(2 * np.pi * s2)  # Eq. (2): NLL = J / σ_Y^2 + N_C ln(2π σ_Y^2)
            res = minimize(nll, popt, method="L-BFGS-B", bounds=list(zip(*bounds)), options={"maxiter":10000})
            popt = res.x if res.success else popt
            jac = create_jacobian(func, Ce, popt)
            rss = np.sum((qe - func(Ce, *popt))**2)
            sigma2 = rss / len(qe) if rss > 0 else 1e-12
            pcov = sigma2 * np.linalg.pinv(jac.T @ jac)
            sd, cv = param_stats(popt, pcov)
            yhat = func(Ce, *popt)
            r2 = r2_score(qe, yhat)
            nll, aic, bic, kic = calculate_metrics(qe, yhat, len(popt), pcov)
            summary[name] = dict(params=popt, r2=r2, nll=nll, aic=aic, bic=bic, kic=kic, names=spec["names"], pcov=pcov)
            for p, s, c, nm in zip(popt, sd, cv, spec["names"]):
                param_table.append(dict(Model=name, Parameter=nm, Value=p, Std_Dev=s, CV_pct=c))
            cov_sheets[name] = pd.DataFrame(pcov, index=spec["names"], columns=spec["names"])
            Ce_s = np.linspace(max(Ce.min()*0.8, 1e-6), Ce.max()*1.1, 200)
            qe_pred = np.clip(func(Ce_s, *popt), 0, None)
            ax_fit.plot(Ce_s, qe_pred, color=spec["color"], ls=spec["ls"], lw=2, label=name)
        except Exception as e:
            print(f"{name} fit failed – {e}")
            summary[name] = dict(r2=np.nan, nll=np.inf, aic=np.inf, bic=np.inf, kic=np.inf)

    aic_v, bic_v, kic_v = [summary[m]["aic"] for m in models], [summary[m]["bic"] for m in models], [summary[m]["kic"] for m in models]
    aic_posterior, bic_posterior, kic_posterior = calculate_posterior_probabilities(aic_v), calculate_posterior_probabilities(bic_v), calculate_posterior_probabilities(kic_v)

    ax_fit.set_xlabel(r"$\mathbf{C_{e}}$ (mg/L)"), ax_fit.set_ylabel(r"$\mathbf{q_{e}}$ (mg/g)"), ax_fit.legend(), fig.tight_layout()
    base, fig_file = os.path.splitext(os.path.basename(file_path))[0], f"{os.path.splitext(os.path.basename(file_path))[0]}_isotherm.png"
    fig.savefig(fig_file, dpi=300, bbox_inches="tight")

    if export_results:
        out = f"{base}_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        with pd.ExcelWriter(out, engine="xlsxwriter") as xl:
            pd.DataFrame({"C0_mgL": c0_vals, "Ce_mgL": Ce, "qe_mgg": qe}).to_excel(xl, "Equilibrium", index=False)
            pd.DataFrame(param_table).to_excel(xl, "Parameters", index=False)
            pd.DataFrame({
                "Model": list(models.keys()), "R2": [summary[m]["r2"] for m in models], "NLL": [summary[m]["nll"] for m in models],
                "AIC": aic_v, "BIC": bic_v, "KIC": kic_v,
                "AIC_Posterior": aic_posterior, "BIC_Posterior": bic_posterior, "KIC_Posterior": kic_posterior,
            }).to_excel(xl, "Model_Comparison", index=False)
            for n, df_cov in cov_sheets.items(): df_cov.to_excel(xl, f"Cov_{n}")
            xl.book.add_worksheet("Plot").insert_image("A1", fig_file)
        print(f"Results written to '{out}' and plot saved to '{fig_file}'.")
    return summary

# Equation (in nll function):
# Eq. (2): NLL = J / σ_Y^2 + N_C ln(2π σ_Y^2)

# ---------------------- 4. CLI -----------------------------------
if __name__ == "__main__":
    analyze_isotherms("Mt-II_Iohexol_Kinetics.xlsx")